In [33]:
import time
import re
import random

bot_template = "BOT : {0}"
user_template = "USER : {0}"

# Define a function that responds to a user's message: respond
def respond(message):
    # Concatenate the user's message to the end of a standard bot respone
    bot_message = "I can hear you! You said: " + message
    # Return the result
    return bot_message

# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

# Send a message to the bot
send_message("hello")


USER : hello
BOT : I can hear you! You said: hello


There are typically two types of characters:

* Ordinary characters - `A`,`b`,`0`
* Specal characters - `[`,`]`,`{`,`}`,`*`,`+`,`?`,`.`,`^` etc.

A very useful link for a deep dive on regex visit the page below:
https://docs.python.org/2/howto/regex.html#regex-howto

In [34]:
#Defining a simple rules dictionary for a set of standard responses
rules ={'I want (.*)': ['What would it mean if you got {0}',
  'Why do you want {0}',
  "What's stopping you from getting {0}"],
 'do you remember (.*)': ['Did you think I would forget {0}',
  "Why haven't you been able to forget {0}",
  'What about {0}',
  'Yes .. and?'],
 'do you think (.*)': ['if {0}? Absolutely.', 'No chance'],
 'if (.*)': ["Do you really think it's likely that {0}",
  'Do you wish that {0}',
  'What do you think about {0}',
  'Really--if {0}']}

# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for pattern,responses in rules.items():
        # Create a match object
        match = re.search(pattern,message)
        if match is not None:
            # Choose a random response
            response = random.choice(responses)
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response, phrase

# Test match_rule
print(match_rule(rules, "do you remember your last birthday"))

('Yes .. and?', None)


In [35]:
# Define replace_pronouns()
def replace_pronouns(message):

    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub("me","you",message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub("my","your",message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub("your","my",message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub("you","me",message)

    return message

print(replace_pronouns("my last birthday"))

your last birthday


In [36]:
# Rewrite respond()
def respond(message):
    # Call match_rule
    response,phrase = match_rule(rules,message)
    if '{0}' in response:
        # Replace the pronouns in the phrase
        phrase = replace_pronouns(phrase)
        # Include the phrase in the response
        response = response.format(phrase)
    return response

# Send the messages
send_message("do you think I am smart?")

USER : do you think I am smart?
BOT : No chance


Recognizing **intents** and **entitites** are the main challenges in coming up with a response. In face **NEP**( Natural Entity Recognition) is in itself a challenging problem in the field of **NLP**

In [37]:
# Intent extraction using regex

# Define a dictionary of patterns
patterns = {}

#keywords dicionary consisting of intents and key words 
keywords= {'goodbye': ['bye', 'farewell'],
 'greet': ['hello', 'hi', 'hey'],
 'thankyou': ['thank', 'thx']}

#dictionary consisting of responses to intetnts
responses={'default': 'default message',
 'goodbye': 'goodbye for now',
 'greet': 'Hello you! :)',
 'thankyou': 'you are very welcome'}

# Iterate over the keywords dictionary
for intent,keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
    patterns[intent] = re.compile('|'.join(keys)) #used to concatenate elements of a list using a separator
    
# Print the patterns
print(patterns)

# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if re.search(pattern,message):#returns a None if no match is found
            matched_intent = intent
    return matched_intent

# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]
#Test
send_message("hello! What's up?")
send_message("thanks very much") 

{'goodbye': re.compile('bye|farewell'), 'greet': re.compile('hello|hi|hey'), 'thankyou': re.compile('thank|thx')}
USER : hello! What's up?
BOT : Hello you! :)
USER : thanks very much
BOT : you are very welcome


#### Entity extraction using regex
Extracting the name from a sentence typically of the form <br/>
"hello, my name is Aravind Atreya"<br/>
So we'll have to look for `"name"` or `"call(ed)"` <br/>
Unless an 'r' or 'R' prefix is present, escape sequences in strings are interpreted according to rules similar to those used by Standard C.<br/>

The 'r' means the the following is a "raw string", ie. backslash characters are treated literally instead of signifying special treatment of the following character. So `'\n'` is a single newline
and `r'\n'` is two characters - a backslash and the letter 'n' 


In [38]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile("name|call")
    # Create a pattern for finding capitalized words
    #capital letter repeating only once and followed by lowercase words, \b s for word boundary
    name_pattern = re.compile(r"\b[A-Z]{1,1}[a-z]*\b")
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Ishmael")
send_message("People call me Cassandra")


USER : my name is David Copperfield
BOT : Hello, David Copperfield!
USER : call me Ishmael
BOT : Hello, Ishmael!
USER : People call me Cassandra
BOT : Hello, People Cassandra!
